# Creating a Deploy App with MONAI Deploy App SDK and MONAI Bundle

This tutorial shows how to create an organ segmentation application for a PyTorch model that has been trained with MONAI and packaged in the [MONAI Bundle](https://docs.monai.io/en/latest/bundle_intro.html) format.

Deploying AI models requires the integration with clinical imaging network, even if in a for-research-use setting. This means that the AI deploy application will need to support standards-based imaging protocols, and specifically for Radiological imaging, DICOM protocol.

Typically, DICOM network communication, either in DICOM TCP/IP network protocol or DICOMWeb, would be handled by DICOM devices or services, e.g. MONAI Deploy Informatics Gateway, so the deploy application itself would only need to use DICOM Part 10 files as input and save the AI result in DICOM Part10 file(s). For segmentation use cases, the DICOM instance file for AI results could be a DICOM Segmentation object or a DICOM RT Structure Set, and for classification, DICOM Structure Report and/or DICOM Encapsulated PDF.

When integrated with imaging networks and receiving DICOM instances from modalities and Picture Archiving and Communications System (PACS), an AI deploy application has to deal with a whole DICOM study with multiple series, whose images' spacing may not be the same as expected by the trained model. To address these cases consistently and efficiently, MONAI Deploy Application SDK provides classes, called operators, to parse DICOM studies, select specific series with application-defined rules, and convert the selected DICOM series into domain-specific image format along with meta-data representing the pertinent DICOM attributes. The image is then further processed in the pre-processing stage to normalize spacing, orientation, intensity,etc, before pixel data as Tensors are used for inference.

In the following sections, we will demonstrate how to create a MONAI Deploy application package using the MONAI Deploy App SDK, and importantly, using the built-in MONAI Bundle Inference Operator to perform inference with the Spleen CT Segmentation PyTorch model in a MONAI Bundle.

:::{note}
For local testing, if there is a lack of DICOM Part 10 files, one can use open source programs, e.g. 3D Slicer, to convert a NIfTI file to a DICOM series.

To make running this example simpler, the DICOM files and the [Spleen CT Segmentation MONAI Bundle](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation), published in [MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo), have been packaged and shared on Google Drive.

:::

## Creating Operators and connecting them in Application class

We will implement an application that consists of five Operators:

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: a folder path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: a list of DICOM studies in memory (List[[`DICOMStudy`](/modules/_autosummary/monai.deploy.core.domain.DICOMStudy)])
    - **Input(selection_rules)**: a selection rule (Dict)
    - **Output(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **MonaiBundleInferenceOperator**:
    - **Input(image)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output(pred)**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: a segmentation image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Input(study_selected_series_list)**: a DICOM series object in memory ([`StudySelectedSeries`](/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries))
    - **Output(dicom_seg_instance)**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))


:::{note}
The `DICOMSegmentationWriterOperator` needs both the segmentation image as well as the original DICOM series meta-data in order to use the patient demographics and the DICOM Study level attributes.
:::

The workflow of the application is illustrated below.

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB

    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list
    DICOMSeriesToVolumeOperator --|> MonaiBundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    MonaiBundleInferenceOperator --|> DICOMSegmentationWriterOperator : pred...seg_image


    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class MonaiBundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### Setup environment


In [1]:
# Install MONAI and other necessary image processing packages for the application
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.12.0"
!python -c "import numpy" || pip install -q "numpy>=1.21.6"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=2.3.0"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"

# Install MONAI Deploy App SDK package
!python -c "import holoscan" || pip install --upgrade -q "holoscan>=0.5.0"
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

# Install Clara Viz package
!python -c "import clara.viz" || pip install --upgrade -q "clara-viz"

Note: you may need to restart the Jupyter kernel to use the updated packages.

### Download/Extract input and model/bundle files from Google Drive

In [2]:
# Download the test data and MONAI bundle zip file
!pip install gdown
!gdown "https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ"

# After downloading ai_spleen_bundle_data zip file from the web browser or using gdown,
!unzip -o "ai_spleen_seg_bundle_data.zip"

Downloading...
From: https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ
To: /home/mqin/src/monai-deploy-app-sdk/notebooks/tutorials/ai_spleen_seg_bundle_data.zip
100%|██████████████████████████████████████| 79.4M/79.4M [00:00<00:00, 80.9MB/s]
Archive:  ai_spleen_seg_bundle_data.zip
  inflating: dcm/1-001.dcm           
  inflating: dcm/1-002.dcm           
  inflating: dcm/1-003.dcm           
  inflating: dcm/1-004.dcm           
  inflating: dcm/1-005.dcm           
  inflating: dcm/1-006.dcm           
  inflating: dcm/1-007.dcm           
  inflating: dcm/1-008.dcm           
  inflating: dcm/1-009.dcm           
  inflating: dcm/1-010.dcm           
  inflating: dcm/1-011.dcm           
  inflating: dcm/1-012.dcm           
  inflating: dcm/1-013.dcm           
  inflating: dcm/1-014.dcm           
  inflating: dcm/1-015.dcm           
  inflating: dcm/1-016.dcm           
  inflating: dcm/1-017.dcm           
  inflating: dcm/1-018.dcm           
  inflating: dcm/1-

### Set up environment variables

In [3]:
%env HOLOSCAN_INPUT_PATH dcm
%env HOLOSCAN_MODEL_PATH model.ts
%env HOLOSCAN_OUTPUT_PATH output
%ls $HOLOSCAN_INPUT_PATH

env: HOLOSCAN_INPUT_PATH=dcm
env: HOLOSCAN_MODEL_PATH=model.ts
env: HOLOSCAN_OUTPUT_PATH=output
1-001.dcm  1-031.dcm  1-061.dcm  1-091.dcm  1-121.dcm  1-151.dcm  1-181.dcm
1-002.dcm  1-032.dcm  1-062.dcm  1-092.dcm  1-122.dcm  1-152.dcm  1-182.dcm
1-003.dcm  1-033.dcm  1-063.dcm  1-093.dcm  1-123.dcm  1-153.dcm  1-183.dcm
1-004.dcm  1-034.dcm  1-064.dcm  1-094.dcm  1-124.dcm  1-154.dcm  1-184.dcm
1-005.dcm  1-035.dcm  1-065.dcm  1-095.dcm  1-125.dcm  1-155.dcm  1-185.dcm
1-006.dcm  1-036.dcm  1-066.dcm  1-096.dcm  1-126.dcm  1-156.dcm  1-186.dcm
1-007.dcm  1-037.dcm  1-067.dcm  1-097.dcm  1-127.dcm  1-157.dcm  1-187.dcm
1-008.dcm  1-038.dcm  1-068.dcm  1-098.dcm  1-128.dcm  1-158.dcm  1-188.dcm
1-009.dcm  1-039.dcm  1-069.dcm  1-099.dcm  1-129.dcm  1-159.dcm  1-189.dcm
1-010.dcm  1-040.dcm  1-070.dcm  1-100.dcm  1-130.dcm  1-160.dcm  1-190.dcm
1-011.dcm  1-041.dcm  1-071.dcm  1-101.dcm  1-131.dcm  1-161.dcm  1-191.dcm
1-012.dcm  1-042.dcm  1-072.dcm  1-102.dcm  1-132.dcm  1-162.dcm  1-

### Set up imports

Let's import necessary classes/decorators to define Application and Operator.

In [4]:

import logging
from pathlib import Path

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.conditions import CountCondition
from monai.deploy.core import AppContext, Application
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.logger import load_env_log_level
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)
from monai.deploy.operators.stl_conversion_operator import STLConversionOperator
from monai.deploy.operators.clara_viz_operator import ClaraVizOperator

### Determining the Input and Output for the Model Bundle Inference Operator

The App SDK provides a `MonaiBundleInferenceOperator` class to perform inference with a MONAI Bundle, which is essentially a PyTorch model in TorchScript with additional metadata describing the model network and processing specification. This operator uses the MONAI utilities to parse a MONAI Bundle to automatically instantiate the objects required for input and output processing as well as inference, as such it depends on MONAI transforms, inferers, and in turn their dependencies.

Each Operator class inherits from the base [Operator](/modules/_autosummary/monai.deploy.core.Operator) class, and its input/output properties are specified by using [@input](/modules/_autosummary/monai.deploy.core.input)/[@output](/modules/_autosummary/monai.deploy.core.output) decorators. For the `MonaiBundleInferenceOperator` class, the input/output need to be defined to match those of the model network, both in name and data type. For the current release, an `IOMapping` object is used to connect the operator input/output to those of the model network by using the same names. This is likely to change, to be automated, in the future releases once certain limitation in the App SDK is removed.

The Spleen CT Segmentation model network has a named input, called "image", and the named output called "pred", and both are of image type, which can all be mapped to the App SDK [Image](/modules/_autosummary/monai.deploy.core.domain.Image). This piece of information is typically acquired by examining the model metadata `network_data_format` attribute in the bundle, as seen in this [example] (https://github.com/Project-MONAI/model-zoo/blob/dev/models/spleen_ct_segmentation/configs/metadata.json).

### Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](/modules/_autosummary/monai.deploy.core.Application) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](/modules/_autosummary/monai.deploy.core.resource) and [@env](/modules/_autosummary/monai.deploy.core.env) decorators.

Objects required for DICOM parsing, series selection, pixel data conversion to volume image, model specific inference, and the AI result specific DICOM Segmentation object writers are created. The execution pipeline, as a Directed Acyclic Graph, is then constructed by connecting these objects through <a href="../../modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

In [5]:
class AISpleenSegApp(Application):
    """Demonstrates inference with built-in MONAI Bundle inference operator with DICOM files as input/output

    This application loads a set of DICOM instances, select the appropriate series, converts the series to
    3D volume image, performs inference with the built-in MONAI Bundle inference operator, including pre-processing
    and post-processing, save the segmentation image in a DICOM Seg OID in an instance file, and optionally the
    surface mesh in STL format.

    Pertinent MONAI Bundle:
      https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation

    Execution Time Estimate:
      With a Nvidia GV100 32GB GPU, for an input DICOM Series of 515 instances, the execution time is around
      25 seconds with saving both DICOM Seg and surface mesh STL file, and 15 seconds with DICOM Seg only.
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        app_context = AppContext({})  # Let it figure out all the attributes without overriding
        app_input_path = Path(app_context.input_path)
        app_output_path = Path(app_context.output_path)
        model_path = Path(app_context.model_path)

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator(
            self, CountCondition(self, 1), input_folder=app_input_path, name="study_loader_op"
        )
        series_selector_op = DICOMSeriesSelectorOperator(self, rules=Sample_Rules_Text, name="series_selector_op")
        series_to_vol_op = DICOMSeriesToVolumeOperator(self, name="series_to_vol_op")

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init.

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            bundle_path=model_path,
            name="bundle_spleen_seg_op",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=segment_descriptions,
            custom_tags=custom_tags,
            output_folder=app_output_path,
            name="dicom_seg_writer",
        )

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {("dicom_study_list", "dicom_study_list")})
        self.add_flow(
            series_selector_op, series_to_vol_op, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {("image", "image")})
        # Note below the dicom_seg_writer requires two inputs, each coming from a source operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {("pred", "seg_image")})
        # Create the surface mesh STL conversion operator and add it to the app execution flow, if needed, by
        # uncommenting the following couple lines.
        stl_conversion_op = STLConversionOperator(
            self, output_file=app_output_path.joinpath("stl/spleen.stl"), name="stl_conversion_op"
        )
        self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {("pred", "image")})

        # Create the ClaraViz operator and feed both the seg and input Image object to it
        viz_op = ClaraVizOperator(self, name="clara_viz_op")
        self.add_flow(series_to_vol_op, viz_op, {("image", "image")})
        self.add_flow(bundle_spleen_seg_op, viz_op, {("pred", "seg_image")})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

## Executing app locally

We can execute the app in the Jupyter notebook. Note that the DICOM files of the CT Abdomen series must be present in the `dcm` and the Torch Script model at `model.ts`. Please use the actual path in your environment.


In [6]:
load_env_log_level()
logging.info(f"Begin {__name__}")
AISpleenSegApp().run()
logging.info(f"End {__name__}")

2023-05-05 12:08:18,085 - Begin __main__
2023-05-05 12:08:18,125 - Begin run
2023-05-05 12:08:18.088 INFO  /workspace/holoscan-sdk/src/core/executors/gxf/gxf_executor.cpp@71: Creating context
2023-05-05 12:08:18,128 - Begin compose
2023-05-05 12:08:18,162 - End compose
2023-05-05 12:08:18.165 INFO  gxf/std/greedy_scheduler.cpp@184: Scheduling 10 entities
2023-05-05 12:08:18,166 - No or invalid input path from the optional input port: None


[2023-05-05 12:08:18.127] [holoscan] [info] [gxf_executor.cpp:100] Loading extensions from configs...
[2023-05-05 12:08:18.163] [holoscan] [info] [gxf_executor.cpp:291] Activating Graph...
[2023-05-05 12:08:18.165] [holoscan] [info] [gxf_executor.cpp:293] Running Graph...
[2023-05-05 12:08:18.165] [holoscan] [info] [gxf_executor.cpp:295] Waiting for completion...


2023-05-05 12:08:18,709 - Finding series for Selection named: CT Series
2023-05-05 12:08:18,711 - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
2023-05-05 12:08:18,711 - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
2023-05-05 12:08:18,712 - On attribute: 'StudyDescription' to match value: '(.*?)'
2023-05-05 12:08:18,712 -     Series attribute StudyDescription value: CT ABDOMEN W IV CONTRAST
2023-05-05 12:08:18,713 - Series attribute string value did not match. Try regEx.
2023-05-05 12:08:18,714 - On attribute: 'Modality' to match value: '(?i)CT'
2023-05-05 12:08:18,714 -     Series attribute Modality value: CT
2023-05-05 12:08:18,715 - Series attribute string value did not match. Try regEx.
2023-05-05 12:08:18,716 - On attribute: 'SeriesDescription' to match value: '(.*?)'
2023-05-05 12:08:18,716 -     Series attribute SeriesDescription value: ABD/PANC 3.0 B31f
2023-05-05 12:08:1

/home/mqin/src/monai-deploy-app-sdk/.venv/lib/python3.8/site-packages/highdicom/valuerep.py:54: UserWarning: The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  warnings.warn(


2023-05-05 12:08:26,887 - add plane #0 for segment #1
2023-05-05 12:08:26,889 - add plane #1 for segment #1
2023-05-05 12:08:26,891 - add plane #2 for segment #1
2023-05-05 12:08:26,893 - add plane #3 for segment #1
2023-05-05 12:08:26,894 - add plane #4 for segment #1
2023-05-05 12:08:26,896 - add plane #5 for segment #1
2023-05-05 12:08:26,898 - add plane #6 for segment #1
2023-05-05 12:08:26,899 - add plane #7 for segment #1
2023-05-05 12:08:26,901 - add plane #8 for segment #1
2023-05-05 12:08:26,902 - add plane #9 for segment #1
2023-05-05 12:08:26,904 - add plane #10 for segment #1
2023-05-05 12:08:26,905 - add plane #11 for segment #1
2023-05-05 12:08:26,907 - add plane #12 for segment #1
2023-05-05 12:08:26,909 - add plane #13 for segment #1
2023-05-05 12:08:26,910 - add plane #14 for segment #1
2023-05-05 12:08:26,912 - add plane #15 for segment #1
2023-05-05 12:08:26,914 - add plane #16 for segment #1
2023-05-05 12:08:26,915 - add plane #17 for segment #1
2023-05-05 12:08:26,

Box(children=(Widget(), VBox(children=(interactive(children=(Dropdown(description='View mode', index=2, option…

2023-05-05 12:08:39.613 INFO  gxf/std/greedy_scheduler.cpp@343: Scheduler stopped: Some entities are waiting for execution, but there are no periodic or async entities to get out of the deadlock.
2023-05-05 12:08:39.613 INFO  gxf/std/greedy_scheduler.cpp@367: Scheduler finished.
2023-05-05 12:08:39,614 - End run
2023-05-05 12:08:39,616 - End __main__


[2023-05-05 12:08:39.613] [holoscan] [info] [gxf_executor.cpp:297] Deactivating Graph...


Once the application is verified inside Jupyter notebook, we can write the above Python code into Python files in an application folder.

The application folder structure would look like below:

```bash
my_app
├── __main__.py
└── app.py
```

In [7]:
# Create an application folder
!mkdir -p my_app

### app.py

In [8]:
%%writefile my_app/app.py

# Copyright 2021-2023 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


import logging
from pathlib import Path

# Required for setting SegmentDescription attributes. Direct import as this is not part of App SDK package.
from pydicom.sr.codedict import codes

from monai.deploy.conditions import CountCondition
from monai.deploy.core import AppContext, Application
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.logger import load_env_log_level
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import (
    BundleConfigNames,
    IOMapping,
    MonaiBundleInferenceOperator,
)
from monai.deploy.operators.stl_conversion_operator import STLConversionOperator
from monai.deploy.operators.clara_viz_operator import ClaraVizOperator


class AISpleenSegApp(Application):
    """Demonstrates inference with built-in MONAI Bundle inference operator with DICOM files as input/output

    This application loads a set of DICOM instances, select the appropriate series, converts the series to
    3D volume image, performs inference with the built-in MONAI Bundle inference operator, including pre-processing
    and post-processing, save the segmentation image in a DICOM Seg OID in an instance file, and optionally the
    surface mesh in STL format.

    Pertinent MONAI Bundle:
      https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation

    Execution Time Estimate:
      With a Nvidia GV100 32GB GPU, for an input DICOM Series of 515 instances, the execution time is around
      25 seconds with saving both DICOM Seg and surface mesh STL file, and 15 seconds with DICOM Seg only.
    """

    def __init__(self, *args, **kwargs):
        """Creates an application instance."""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # This method calls the base class to run. Can be omitted if simply calling through.
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """Creates the app specific operators and chain them up in the processing DAG."""

        logging.info(f"Begin {self.compose.__name__}")

        app_context = AppContext({})  # Let it figure out all the attributes without overriding
        app_input_path = Path(app_context.input_path)
        app_output_path = Path(app_context.output_path)
        model_path = Path(app_context.model_path)

        # Create the custom operator(s) as well as SDK built-in operator(s).
        study_loader_op = DICOMDataLoaderOperator(
            self, CountCondition(self, 1), input_folder=app_input_path, name="study_loader_op"
        )
        series_selector_op = DICOMSeriesSelectorOperator(self, rules=Sample_Rules_Text, name="series_selector_op")
        series_to_vol_op = DICOMSeriesToVolumeOperator(self, name="series_to_vol_op")

        # Create the inference operator that supports MONAI Bundle and automates the inference.
        # The IOMapping labels match the input and prediction keys in the pre and post processing.
        # The model_name is optional when the app has only one model.
        # The bundle_path argument optionally can be set to an accessible bundle file path in the dev
        # environment, so when the app is packaged into a MAP, the operator can complete the bundle parsing
        # during init.

        config_names = BundleConfigNames(config_names=["inference"])  # Same as the default

        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            self,
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
            app_context=app_context,
            bundle_config_names=config_names,
            bundle_path=model_path,
            name="bundle_spleen_seg_op",
        )

        # Create DICOM Seg writer providing the required segment description for each segment with
        # the actual algorithm and the pertinent organ/tissue. The segment_label, algorithm_name,
        # and algorithm_version are of DICOM VR LO type, limited to 64 chars.
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.3.2",
            )
        ]

        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            self,
            segment_descriptions=segment_descriptions,
            custom_tags=custom_tags,
            output_folder=app_output_path,
            name="dicom_seg_writer",
        )

        # Create the processing pipeline, by specifying the source and destination operators, and
        # ensuring the output from the former matches the input of the latter, in both name and type.
        self.add_flow(study_loader_op, series_selector_op, {("dicom_study_list", "dicom_study_list")})
        self.add_flow(
            series_selector_op, series_to_vol_op, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {("image", "image")})
        # Note below the dicom_seg_writer requires two inputs, each coming from a source operator.
        self.add_flow(
            series_selector_op, dicom_seg_writer, {("study_selected_series_list", "study_selected_series_list")}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {("pred", "seg_image")})
        # Create the surface mesh STL conversion operator and add it to the app execution flow, if needed, by
        # uncommenting the following couple lines.
        stl_conversion_op = STLConversionOperator(
            self, output_file=app_output_path.joinpath("stl/spleen.stl"), name="stl_conversion_op"
        )
        self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {("pred", "image")})

        # Create the ClaraViz operator and feed both the seg and input Image object to it
        viz_op = ClaraVizOperator(self, name="clara_viz_op")
        self.add_flow(series_to_vol_op, viz_op, {("image", "image")})
        self.add_flow(bundle_spleen_seg_op, viz_op, {("pred", "seg_image")})

        logging.info(f"End {self.compose.__name__}")


# This is a sample series selection rule in JSON, simply selecting CT series.
# If the study has more than 1 CT series, then all of them will be selected.
# Please see more detail in DICOMSeriesSelectorOperator.
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

Overwriting my_app/app.py


```python
if __name__ == "__main__":
    AISpleenSegApp().run()
```

The above lines are needed to execute the application code by using `python` interpreter.

### \_\_main\_\_.py

\_\_main\_\_.py is needed for <a href="../../developing_with_sdk/packaging_app.html#required-arguments">MONAI Application Packager</a> to detect the main application code (`app.py`) when the application is executed with the application folder path (e.g., `python simple_imaging_app`).

In [13]:
%%writefile my_app/__main__.py
from app import AISpleenSegApp

import logging

if __name__ == "__main__":
    logging.info("test")
    AISpleenSegApp().run()

Overwriting my_app/__main__.py


In [10]:
!ls my_app

app.py	__main__.py  __pycache__  spleen_seg_operator.py


In this time, let's execute the app in the command line.

In [11]:
!python my_app

2023-05-05 12:31:39.129 INFO  /workspace/holoscan-sdk/src/core/executors/gxf/gxf_executor.cpp@71: Creating context
2023-05-05 12:31:39,152 - Begin run
[2023-05-05 12:31:39.153] [holoscan] [info] [gxf_executor.cpp:100] Loading extensions from configs...
2023-05-05 12:31:39,153 - Begin compose
2023-05-05 12:31:39,184 - End compose
[2023-05-05 12:31:39.185] [holoscan] [info] [gxf_executor.cpp:291] Activating Graph...
[2023-05-05 12:31:39.186] [holoscan] [info] [gxf_executor.cpp:293] Running Graph...
[2023-05-05 12:31:39.186] [holoscan] [info] [gxf_executor.cpp:295] Waiting for completion...
2023-05-05 12:31:39.186 INFO  gxf/std/greedy_scheduler.cpp@184: Scheduling 10 entities
2023-05-05 12:31:39,186 - No or invalid input path from the optional input port: None
2023-05-05 12:31:39,565 - Finding series for Selection named: CT Series
2023-05-05 12:31:39,565 - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
2023-05-05 12:31:39,566 - Working

In [12]:
!ls output

1.2.826.0.1.3680043.10.511.3.12261238603895552550823675908544610.dcm
1.2.826.0.1.3680043.10.511.3.12630758166938890127403819248199119.dcm
1.2.826.0.1.3680043.10.511.3.13096597901857940207245232021669675.dcm
1.2.826.0.1.3680043.10.511.3.13179553861865025383226235853759453.dcm
1.2.826.0.1.3680043.10.511.3.17388926533738991308007815250566978.dcm
1.2.826.0.1.3680043.10.511.3.23793490991542835187547578029858598.dcm
1.2.826.0.1.3680043.10.511.3.30270186454698610203395168618664023.dcm
1.2.826.0.1.3680043.10.511.3.30490464162936274579388472251736399.dcm
1.2.826.0.1.3680043.10.511.3.35280861260524240908880539194328489.dcm
1.2.826.0.1.3680043.10.511.3.36898062718290332296841229400741012.dcm
1.2.826.0.1.3680043.10.511.3.50905662781411471396234178786271734.dcm
1.2.826.0.1.3680043.10.511.3.58695380375390884831286990782807630.dcm
1.2.826.0.1.3680043.10.511.3.59477167967116171518347083763624890.dcm
1.2.826.0.1.3680043.10.511.3.63882181243080191729094928676268998.dcm
1.2.826.0.1.3680043.10.511.3.77716

## Packaging app

Clara-Viz operators added in an application are used for interactive visualization, so the application shall not be packaged.